In [369]:
from bs4 import BeautifulSoup 
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
# from selenium import webdriver
from urllib.parse import urlencode
import json
import os
import pandas as pd
import re
import requests
import time
from datetime import datetime
import math
import random
from fake_useragent import UserAgent

In [6]:
url = "https://rent.591.com.tw/?kind=0&region=1"

In [7]:
r = requests.get(url,verify = False)

d:\programfiles\python37\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [10]:
soup = BeautifulSoup(r.text,"html.parser")

In [57]:
region_map = {
    '台北市': '1',
    '基隆市': '2',
    '新北市': '3',
    '新竹市': '4',
    '新竹縣': '5',
    '桃園市': '6',
    '苗栗縣': '7',
    '台中市': '8',
    '彰化縣': '10',
    '南投縣': '11',
    '嘉義市': '12',
    '嘉義縣': '13',
    '雲林縣': '14',
    '台南市': '15',
    '高雄市': '17',
    '屏東縣': '19',
    '宜蘭縣': '21',
    '台東縣': '22',
    '花蓮縣': '23',
    '澎湖縣': '24',
    '金門縣': '25',
    '連江縣': '26',
}

In [279]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}
sess = requests.session()

In [249]:
##獲取換頁之驗證參數
def _get_val(sess, region, headers):
    base_url = "https://rent.591.com.tw/?kind=0&regionid=" + region_map[region]
    res = sess.get(base_url, headers = headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp']= region_map[region]
#     print(res.cookies.get_dict())
    soup = BeautifulSoup(res.text, "lxml")
    #     print(soup.select('head > meta'))

    totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[['data-total']
#     print(totalRows)
    csrf_token = [i['content'] for i in soup.select('head > meta[name]') if i['name'] == "csrf-token"][0]

    return (totalRows, csrf_token)

In [485]:
r = requests.get("https://rent.591.com.tw/?kind=0&regionid=1")
soup = BeautifulSoup(r.text)
soup.select("#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext")[0]["data-total"]

'13212'

In [266]:
#同一頁面所有詳細項目網址
# url = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow%3D=240&totalRows=380"
# # url = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=30&totalRows=380"
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
#     "X-CSRF-TOKEN": "wIcwCSLpk3yuCByqfVrcYkPRPsFHyR62pnvtwOoM"
# }
# params = {
#     "is_new_list": "1",
#     "type": "1",
#     "kind": "0",
#     "searchtype": "1",
#     "region": "23",
#     "firstRow": "30",
#     "totalRows": "380"
# }
# resp = requests.get(url, headers = params)
# resp.text
# def _get_url(resp):
# soup = BeautifulSoup(resp.text,"html.parser")
# pattern = r"html$"
# href = soup.find("div",{"id":"content"}) #.find_all("a")
# print(href)
# url = ["https:" + h["href"] for h in href]
# url = [u for u in url if re.search(pattern, u)]
#     return url
# _get_url(resp)

'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n\n        <title>Page Expired</title>\n\n        <!-- Fonts -->\n        <link href="https://fonts.googleapis.com/css?family=Raleway:100,600" rel="stylesheet" type="text/css">\n\n        <!-- Styles -->\n        <style>\n            html, body {\n                background-color: #fff;\n                color: #636b6f;\n                font-family: \'Raleway\', sans-serif;\n                font-weight: 100;\n                height: 100vh;\n                margin: 0;\n            }\n\n            .full-height {\n                height: 100vh;\n            }\n\n            .flex-center {\n                align-items: center;\n                display: flex;\n                justify-content: center;\n            }\n\n            .position-ref {\n                position: re

In [284]:
vars = _get_val(sess, '花蓮縣', headers)

In [285]:
vars

('377', 'CeHh3Sb95bJ3729gYwubpaJPYJ1J8xdP1s8DzA5M')

In [276]:
# headers.update({'X-CSRF-Token':vars[1]})
# del headers["X-CSRF-Token"]


In [365]:
#獲取全部頁面url
def _get_allurl_list(sess, region, headers):
    val = _get_val(sess,region,headers)
    headers.update({'X-CSRF-TOKEN':val[1]})
    print(f"headers:{headers}")
    times = math.floor(int(val[0])/30) +1 if int(val[0])%30 !=0 else math.floor(int(val[0])/30)
#     searchurlist=[]
    for firstRow in range(times):
        searchurl = _get_rent_list(firstRow,region,headers)
        print(searchurl)
        res_search = sess.get(searchurl, headers=headers)
        res_search.encoding = 'utf-8'
        data = json.loads(res_search.text)
#         print(data['data']['data'][0]['post_id'])
        for i in data['data']['data']:
            yield i['post_id'],i['region_name'],i['nick_name'],i['region_name']+' '+i['fulladdress'],'https://rent.591.com.tw/rent-detail-'+str(i['post_id'])+'.html'
        time.sleep(random.randint(1,3))
        print(f"next page------------------------------------------------------------------") 
#             yield rent_dict
#         print(rent_list[0])
#         searchurlist.extend(rent_dict)     
        
    print(f"the end------------------------------------------------------------------------")    

In [297]:
# _get_allurl_list(sess, "花蓮縣", headers)

headers:{'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 'X-CSRF-TOKEN': 'CeHh3Sb95bJ3729gYwubpaJPYJ1J8xdP1s8DzA5M'}
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow=0&totalRows=377
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow=30&totalRows=377
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow=60&totalRows=377
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow=90&totalRows=377
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow=120&totalRows=377
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&sear

[[8984926,
  '花蓮縣',
  '屋主 邱先生',
  '花蓮縣 國興二街花蓮車站前全新豪華套房出租',
  'https://rent.591.com.tw/rent-detail-8984926.html'],
 [8971524,
  '花蓮縣',
  '屋主 蔡先生',
  '花蓮縣 國盛五街降租站前市區有陽台便捷舒適安全好樓（套',
  'https://rent.591.com.tw/rent-detail-8971524.html'],
 [8999527,
  '花蓮縣',
  '代理人 李小姐',
  '花蓮縣 自強路自強路全新電梯獨立套房',
  'https://rent.591.com.tw/rent-detail-8999527.html'],
 [8944645,
  '花蓮縣',
  '屋主 呂小姐',
  '花蓮縣 南京街全新裝潢1+1房電梯大樓包水包網路',
  'https://rent.591.com.tw/rent-detail-8944645.html'],
 [8935215,
  '花蓮縣',
  '屋主 羅小姐',
  '花蓮縣 延平街148巷正市區。近花商、85度c',
  'https://rent.591.com.tw/rent-detail-8935215.html'],
 [8931708,
  '花蓮縣',
  '屋主 邱先生',
  '花蓮縣 建中街電梯大套房,近車站,通風窗採光佳',
  'https://rent.591.com.tw/rent-detail-8931708.html'],
 [8984331,
  '花蓮縣',
  '屋主 楊小姐',
  '花蓮縣 國興一街近花蓮火車站,生活機能佳!',
  'https://rent.591.com.tw/rent-detail-8984331.html'],
 [8993902,
  '花蓮縣',
  '屋主 呂小姐',
  '花蓮縣 中美十一街全新整理大3房2廳2衛有車位可短租',
  'https://rent.591.com.tw/rent-detail-8993902.html'],
 [9023493,
  '花蓮縣',
  '屋主 王先生',
  '花蓮縣 明仁一街185巷不看會後悔!陽台大套房出租',
  'https:

In [299]:
s = '仲介 王先生'
s.split()

['仲介', '王先生']

In [417]:
#main function
def _crawl(region):
    ua = UserAgent()
    user_agent_list = list()
    for i in range(50):
        user_agent_list.append(ua.random)
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    rent_list = []
    with requests.Session() as sess:
        headers['user-agent'] = random.choice(user_agent_list)
        for post_id, region_name, nick_name, fulladdress, url in _get_allurl_list(sess, region, headers):
            rent_out = _get_rent_info(url, sess)
            identification, name = nick_name.split()[0],nick_name.split()[1] 
            update_dict = {"id":post_id, "region":region_name,"出租者":name,"出租者身份":identification,"地址":fulladdress}
            rent_out.update(update_dict)
            rent_list.append(rent_out)
    return rent_list
#             rent_out.update({"id":post_id})
        
#             logger.info(f'[{self.crawler_name}] Get category_url: {category_url}')
#             campaigns = self._get_category_campaigns(category_url, sess)
#             time.sleep(5)
        

In [419]:
%time _crawl("宜蘭縣")

headers:{'user-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/533.19.4 (KHTML, like Gecko) Version/5.0.2 Safari/533.18.5', 'X-CSRF-TOKEN': 'KMwk7OrsMbxciBr6lPQ5vWo8FVT2MuAeUQjwGAhb'}
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E5%AE%9C%E8%98%AD%E7%B8%A3&firstRow=0&totalRows=595
next page------------------------------------------------------------------
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E5%AE%9C%E8%98%AD%E7%B8%A3&firstRow=30&totalRows=595
next page------------------------------------------------------------------
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E5%AE%9C%E8%98%AD%E7%B8%A3&firstRow=60&totalRows=595
next page------------------------------------------------------------------
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E5%AE%9C%E8%98%AD%E7%B8%A3&firstRow=90&to

[{'url': 'https://rent.591.com.tw/rent-detail-8989232.html',
  'region': '宜蘭縣',
  '出租者': '陳小姐',
  '出租者身份': '屋主',
  '聯絡電話': '0986-851-077 轉 1361915',
  '租金': '10,000 元/月',
  '型態': '別墅',
  '現況': '獨立套房',
  '性別要求': '女生',
  'update_time': '2020-03-28T21:45:48.559127',
  'id': 8989232,
  '地址': '宜蘭縣 大德路二段<機會難得>別墅高級套房出租'},
 {'url': 'https://rent.591.com.tw/rent-detail-8993063.html',
  'region': '宜蘭縣',
  '出租者': '周先生',
  '出租者身份': '屋主',
  '聯絡電話': '0982-160-445',
  '租金': '5,500 元/月',
  '型態': '別墅',
  '現況': '分租套房',
  '性別要求': '男女生皆可',
  'update_time': '2020-03-28T21:45:48.882006',
  'id': 8993063,
  '地址': '宜蘭縣 擺厘路9巷宜蘭大學精美獨立陽台套房'},
 {'url': 'https://rent.591.com.tw/rent-detail-8982717.html',
  'region': '宜蘭縣',
  '出租者': '周先生',
  '出租者身份': '屋主',
  '聯絡電話': '0982-160-445',
  '租金': '5,500 元/月',
  '型態': '透天厝',
  '現況': '分租套房',
  '性別要求': '男女生皆可',
  'update_time': '2020-03-28T21:45:49.220102',
  'id': 8982717,
  '地址': '宜蘭縣 進士路151巷宜蘭大學精美套房'},
 {'url': 'https://rent.591.com.tw/rent-detail-9009342.html',
  'region

In [ ]:
# def _crawl(self):
#     region = 1
    
#     with requests.Session() as sess:
       
#         totalRows, csrf_token = _get_val(sess, region, headers)
#         region_url = _get_rent_list(region, totalRows)
        
#         for region_url in region_urls:
# #             logger.info(f'[{self.crawler_name}] Get category_url: {category_url}')
# #             campaigns = self._get_category_campaigns(category_url, sess)
# #             time.sleep(5)
#             yield campaigns

In [ ]:
# def _get_offer_pages(self, category_url, sess):  # 獲取頁面資訊list of dict(第幾頁)
#     out = [
#         {
#             "url": _get_rent_list(??),
#             "headers": {
#                 "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
#                 "content-type": "application/x-www-form-urlencoded",
#                 "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
#             },
#         }
#         for i in range(1, totalRows + 1)
#     ]
#     return out

In [328]:
def _get_rent_info(rent_url, sess):  # (self, rent_url, sess):
    if sess is None:
        r = requests.Session()
        rent_page = r.get(rent_url, headers=headers, verify=False)
    else:
        rent_page = sess.get(rent_url, headers=sess.headers, verify=False)
    page = BeautifulSoup(rent_page.text, 'html.parser')
    url = rent_url
    phone = _get_phone(page)
    money, form, situation = _get_description(page)
    gender = _get_condition(page)
#     card_description = self._get_description(page)
#     benefits = self._get_benefits(page)
#     campaigns = self._get_campaigns(page)
    out = {
        "url": url,
        "region": "",
        "出租者":"", 
        "出租者身份":"", 
        "聯絡電話":phone,
        "租金":money,
        "型態":form,
        "現況":situation,
        "性別要求":gender,
        "update_time": datetime.now().isoformat()
    }
    return out

In [224]:
sess = requests.session()
_get_rent_info("https://rent.591.com.tw/rent-detail-8984926.html", sess)

d:\programfiles\python37\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'url': 'https://rent.591.com.tw/rent-detail-8984926.html',
 'region': '',
 '出租者': '邱先生',
 '出租者身份': '屋主',
 '聯絡電話': '0928-080-877',
 '租金': '10,000 元/月',
 '型態': '電梯大樓',
 '現況': '獨立套房',
 '性別要求': '男女生皆可',
 'update_time': '2020-03-28T12:09:41.822409'}

In [118]:
test_url = "https://rent.591.com.tw/rent-detail-8984926.html"

In [119]:
resp = requests.get(test_url,verify = False)

d:\programfiles\python37\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [120]:
pprint(resp.text)

('<!DOCTYPE html>\n'
 '<html xmlns="http://www.w3.org/1999/xhtml" '
 'xmlns:fb="http://www.facebook.com/2008/fbml">\n'
 '<head>\n'
 '    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n'
 '    <title>獨立套房出租,花蓮車站前全新豪華套房出租-花蓮花蓮市房屋出租-591租屋網</title>\n'
 '    <meta name="keywords" '
 'content="591,591租房網,租屋,租屋網,591出租網,獨立套房出租,花蓮租屋,花蓮市租屋,租房網,591租屋網,房屋出租,519" />\n'
 '    <meta name="description" '
 'content="591租房網,花蓮花蓮市租屋資訊:出租獨立套房,璽鈺之星,花蓮車站前全新豪華套房出租,1.環境高雅採光良好鬧中取靜步行即可到達花蓮火車站、中山運動公園、健身房、電影.." '
 '/>\n'
 '    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n'
 '    <meta http-equiv="content-language" content="zh-tw">\n'
 '    <meta name="author" content="數字科技股份有限公司" />\n'
 '    <meta name="copyright" content="www.591.com.tw" />\n'
 '    <meta name="viewport" '
 'content="width=device-width,initial-scale=1,user-scalable=1" />\n'
 '    <meta name="robots" content="ALL" />\n'
 '    <meta property="og:type" content="company"/>\n'
 '    <meta property="og:title"

In [351]:
def _get_phone(resp):
#     soup = BeautifulSoup(resp.text)
#     landlord = soup.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.userInfo > div.infoOne.clearfix > div.avatarRight > div")
#     pos = landlord[0].get_text().find("屋主") if  landlord[0] is not None else -1
#     name, identification = landlord[0].get_text()[:pos-1], landlord[0].get_text()[pos:pos + 2] if pos != -1 else ""
    try:
        phone = resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.userInfo > div:nth-child(2) > span.dialPhoneNum")[0]["data-value"] if resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.userInfo > div:nth-child(2) > span.dialPhoneNum") != [] else ""
    except:
        phone = ""
    return phone

In [141]:
_find_landlord(resp)

('邱先生', '屋主')

In [352]:
def _get_description(resp):
    money = resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > div.price.clearfix")[0].get_text().strip() if resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > div.price.clearfix") != [] else "" 
    status = resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > ul")[0].get_text() if resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > ul") != [] else ""
    try:
        form = ("".join(status[status.find("型態") + 2 : status.find("現況")].split())).split(":")[1] if status != [] else ""
        situation = ("".join(status[status.find("現況") + 2 : status.find("社區")].split())).split(":")[1] if status != [] else ""
    except:
        form = ""
        situation = ""
    return money, form, situation

In [177]:
status = soup.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > ul")[0].get_text()

In [186]:
form = ("".join(status[status.find("型態") + 2 : status.find("現況")].split())).split(":")[1]

'電梯大樓'

In [188]:
situation = ("".join(status[status.find("現況") + 2 : status.find("社區")].split())).split(":")[1]

'獨立套房'

In [480]:
def _get_condition(resp):
    try:
        condition = resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.leftBox > ul.clearfix.labelList.labelList-1")[0].get_text() if resp.select("#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.leftBox > ul.clearfix.labelList.labelList-1") != [] else ""
#         print(condition)
        gender = (condition[condition.find("性別要求")+ 4:condition.find("性別要求")+ 10]).split("：")[1] if condition != [] else ""
#         print(gender)
        check_list = ['男女生皆可', '男生','男性','女生','女性',]
        for i in check_list:
            if gender.find(i) != -1:
                gender = i
                break
            else:
                pass
#         if gender not in check_list:
            
    except:
        gender = ""
    return gender

In [481]:
# gender = (condition[condition.find("性別要求")+ 4:condition.find("朝向")]).split("：")[1]

In [482]:
test_url = "https://rent.591.com.tw/rent-detail-8984926.html"

In [483]:
resp = requests.get(test_url)
soup = BeautifulSoup(resp.text)

In [484]:
_get_condition(soup)

'男女生皆可'

In [ ]:
# url = 'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=0&totalRows=394'

In [79]:
# url = 'https://rent.591.com.tw/rent-detail-8935215.html'

In [83]:
# res1 = sess.get(url, headers=headers)

In [85]:
# res1.encoding = 'utf-8'

In [86]:
# soup_detail = BeautifulSoup(res_detail.text, "lxml")

In [295]:
#獲取某縣市所有頁面url
def _get_rent_list(firstRow,region,headers):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region,
        'firstRow':(firstRow*30),
        'totalRows':_get_val(sess,region,headers)[0]
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
#     https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=%E8%8A%B1%E8%93%AE%E7%B8%A3&firstRow%3D=0&totalRows=377
#     url = base_url + "is_new_list=" + str(params["is_new_list"]) + "&type=" + str(params["type"]) + "&kind=" + str(params["kind"]) + "&searchtype=" + str(params["searchtype"]) + "&region=" + str(params["region"]) + "&firstRow=" + str(params["firstRow"]) + "&totalRows=" + str(params["totalRows"])
    url = base_url + urlencode(params)
    return url

In [115]:
search_url = _get_rent_list(1)
search_url

'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1&firstRow%3D=30&totalRows=383'

In [93]:
res_search = sess.get(search_url,headers=headers)

In [103]:
res_search.encoding = 'utf-8'

In [109]:
# res_search.text
data = json.loads(res_search.text)
# data

In [117]:
#只有30筆資料
data["data"]["data"]

[{'id': 8894447,
  'user_id': 1267458,
  'address': '民族路全新電梯大樓..',
  'type': '1',
  'post_id': 8894447,
  'regionid': 23,
  'sectionid': 358,
  'streetid': 34935,
  'room': 0,
  'area': 10,
  'price': '8,500',
  'storeprice': 0,
  'comment_total': 2,
  'comment_unread': 2,
  'comment_ltime': 1585135490,
  'hasimg': 1,
  'kind': 3,
  'shape': 3,
  'houseage': 0,
  'posttime': '6分鐘內',
  'updatetime': 1582897968,
  'refreshtime': 1585295702,
  'checkstatus': 1,
  'status': '',
  'closed': 0,
  'living': 'depart,advstore,market,park,school,hospital',
  'condition': 'tv,icebox,cold,washer,hotwater,broadband,lift,landpost,cook,balcony_1,bed,wardrobe,bookTable,chair,all_sex',
  'isvip': 1,
  'mvip': 1,
  'is_combine': 1,
  'cover': 'https://hp2.591.com.tw/house/active/2020/02/28/158289817388289105_210x158.crop.jpg',
  'browsenum': 29,
  'browsenum_all': 973,
  'floor2': 0,
  'floor': 2,
  'ltime': '2020-03-27 16:00:05',
  'cases_id': 0,
  'social_house': 0,
  'distance': 0,
  'search_name': '

In [420]:
region = ['台北市', '基隆市', '新北市', '新竹市', '新竹縣', '桃園市', '苗栗縣', '台中市', '彰化縣', '南投縣', '嘉義市', '嘉義縣', '雲林縣', '台南市', '高雄市', '屏東縣', '宜蘭縣', '台東縣', '花蓮縣', '澎湖縣', '金門縣', '連江縣']
house_jobs = [f"{r}" for r in region]
house_jobs

['台北市',
 '基隆市',
 '新北市',
 '新竹市',
 '新竹縣',
 '桃園市',
 '苗栗縣',
 '台中市',
 '彰化縣',
 '南投縣',
 '嘉義市',
 '嘉義縣',
 '雲林縣',
 '台南市',
 '高雄市',
 '屏東縣',
 '宜蘭縣',
 '台東縣',
 '花蓮縣',
 '澎湖縣',
 '金門縣',
 '連江縣']

In [486]:
check_list = ['男女生皆可', '男生', '男性', '女生', '女性']

In [488]:
test = "男女生皆可"
if test not in check_list:
    print("check it out")
else:
    print("good")

good


In [494]:
def _get_val(sess, region, headers):
    base_url = "https://rent.591.com.tw/?kind=0&regionid=" + region
    res = sess.get(base_url, headers=headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp'] = region
#     print(res.cookies.get_dict())
    soup = BeautifulSoup(res.text, "lxml")
#     print(soup.select('head > meta'))
    try:
        totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total'] if soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext') != [] else soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a:nth-child(16)')[0]['data-total']
    except Exception as e:
        print(e)
        totalRows = soup.find("a", {"class": "pageNum-form"})["data-total"] if soup.find("a", {"class": "pageNum-form"}) is not None else ""
    csrf_token = [i['content'] for i in soup.select('head > meta[name]') if i['name'] == "csrf-token"][0] if [i['content'] for i in soup.select('head > meta[name]') if i['name'] == "csrf-token"] != [] else []

    exc_resp = sess.get("https://rent.591.com.tw/?kind=0&regionid=" + region)
    soup_exc = BeautifulSoup(exc_resp.text, 'html.parser')
    if csrf_token == []:
        csrf_token = [i['content'] for i in soup_exc.select('head > meta[name]') if i['name'] == "csrf-token"][0]
    if totalRows == "":
        totalRows = soup_exc.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']
    return (totalRows, csrf_token)

In [495]:
_get_val(sess, "1", headers)

('13207', 'ygrr6MPdGIWsocwWDE2ItlqPcpT1GdkusA2KxvJk')

In [496]:
headers

{'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
 'X-CSRF-TOKEN': 'CeHh3Sb95bJ3729gYwubpaJPYJ1J8xdP1s8DzA5M'}

In [508]:
url = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1&firstRow=1500&totalRows=13219"

In [509]:
r = requests.session().get(url)

In [510]:
soup0330 = BeautifulSoup(r.text,"html.parser")

In [516]:
soup0330.find("meta",{"name":"csrf-token"})["content"]
# soup0330.select('head')

TypeError: 'NoneType' object is not subscriptable

In [518]:
soup0330.select('#container')  # > section.listBox > div > div.listLeft > div.page-limit > div > a:nth-child(16)')

[]

In [519]:
t_list = []
t = ("1","2")
t_list.append(t)


In [520]:
t_list

[('1', '2')]

In [521]:
t1 = ([0],"4")
t_list.append(t1)
t_list

[('1', '2'), ([0], '4')]

In [522]:
for i in t_list:
    if i == ([0], '4'):
        print("number:",i)

number: ([0], '4')


In [524]:
t_list[1]

([0], '4')

In [566]:
pattern = r"=$"

In [567]:
searchurl = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1&firstRow=1500&totalRows="


In [570]:
searchurl = searchurl + "13270" if re.search(pattern, searchurl) else searchurl


In [571]:
searchurl

'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1&firstRow=1500&totalRows=13270'